# Description
This notebook merges YOLO predictions with manual annotations into a unified dataset. Corrected predictions are added to the training pool for subsequent cycles.

# Libraries

In [ ]:
from src import (
    load_metadata,
    copy_frames,
    get_image_size,
    yolo_txt_to_annotation_json,
    save_metadata
)

import os

# Main

In [ ]:
source_dir = ""
al_dir = ""

In [ ]:
predict_dir = os.path.join(al_dir, "predict")

annotations_dir = os.path.join(al_dir, "annotations")
annotations_json = "annotation.json"

In [ ]:
# Load and combine metadata for copying frames
predict_metadata = load_metadata(
    source_dir=al_dir,
    metadata_filename="predict.json"
)

manual_annotation_metadata = load_metadata(
    source_dir=al_dir,
    metadata_filename="annotations_metadata.json"
)

# combine lists
manual_annotation_metadata.extend(predict_metadata)

#### Copy frames referenced in combined annotation metadata into activeLearning_dir/annotations

In [ ]:

split_data_combine = (
    ("annotations", "annotations_metadata.json"),
    (manual_annotation_metadata, True),
    (source_dir, al_dir),
)

copy_frames(
    split=split_data_combine
)

#### Load the current manual annotations JSON (will add predictions into it)

In [ ]:
manual_annotation_json_metadata = load_metadata(
    source_dir=annotations_dir,
    metadata_filename=annotations_json
)

#### Convert YOLO .txt predictions in predict_dir into internal JSON annotation entries

In [ ]:
manually_annotated_flag = False
visible_percentage = 0.90
keypoint_names = ["nose", "earL", "earR", "tailB"]

In [ ]:
# Iterate over prediction label files in predict_dir
for label in os.listdir(predict_dir):
    if label.endswith(".txt"):
        label_path = os.path.join(predict_dir, label)

        # Find corresponding image filename by swapping extension
        image_name = label[:-4] + ".jpg"  # strictly replace '.txt' → '.jpg'

        img_path = os.path.join(predict_dir, image_name)
        if not os.path.exists(img_path):
            # If the matching image isn't present, skip gracefully
            continue

        img_w, img_h = get_image_size(image_path=img_path)

        preds = yolo_txt_to_annotation_json(
            txt_path=label_path,
            image_filename=image_name,
            image_width=img_w,
            image_height=img_h,
            manually_annotated_flag=manually_annotated_flag,
            visible_percentage=visible_percentage,
            keypoint_names=keypoint_names,
        )

        # Merge predictions for this image into the combined JSON
        # (update will add new key or overwrite existing with new list)
        if image_name in manual_annotation_json_metadata:
            manual_annotation_json_metadata[image_name].extend(
                preds.get(image_name, [])
            )
        else:
            manual_annotation_json_metadata.update(preds)

In [ ]:
# Save updated combined annotations
save_metadata(
    output_dir=annotations_dir, 
    metadata_filename="annotation.json", 
    metadata=manual_annotation_json_metadata
)

Back to step 5 [`/al_s5_p1(post-training)_anno-vs-al_split.ipynb`](/al_s5_p1(post-training)_anno-vs-al_split.ipynb), if the desired results are not there.